In [ ]:
!pip install -q -U \
  "bitsandbytes==0.46.0" \
  "transformers==4.41.2" \
  "peft==0.11.1" \
  "accelerate==0.31.0" \
  "datasets==2.19.2" \
  "trl==0.8.6" \
  "huggingface_hub" \
  "minijinja" \
  "triton==3.2.0"

In [ ]:
import random
import numpy as np
import torch

def fix_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
from huggingface_hub import login
import os

login()

In [ ]:

!unzip Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1.zip

print("✅ Arquivo descompactado com sucesso!")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

# Parâmetros de Configuração

# O ID do modelo base original
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# O caminho para a pasta do seu adaptador salvo.
ADAPTER_PATH = "Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1"



print(f"Carregando o modelo base: {base_model_id}")


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)


print(f"\nCarregando o adaptador LoRA de: {ADAPTER_PATH}")

try:
    # Carrega o adaptador e o acopla ao modelo base
    finetuned_model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

    print("\nOtimizando o modelo para inferência...")

    finetuned_model = finetuned_model.merge_and_unload()

    print("✅ Modelo fine-tuned pronto para avaliação!")

except Exception as e:
    print(f"❌ Erro ao carregar o adaptador. Verifique se o caminho '{ADAPTER_PATH}' está correto.")
    print(f"   Erro: {e}")
    finetuned_model = None

In [ ]:


# Parâmetros de Configuração da Avaliação


DEV_FILE = 'dev_formatted.jsonl'

# Nome do arquivo para salvar os resultados do modelo fine-tuned
LOG_FILE_FINETUNED_MODEL = 'evaluation_results_finetuned_model.jsonl'

# Quantidade de exemplos para testar
NUM_EXAMPLES_TO_TEST = 200

if finetuned_model:
    print("\n--- Iniciando avaliação do MODELO FINE-TUNED ---")
    evaluate_model(
        model=finetuned_model,
        tokenizer=tokenizer,
        dev_dataset_path=DEV_FILE,
        output_log_file=LOG_FILE_FINETUNED_MODEL,
        num_examples=NUM_EXAMPLES_TO_TEST
    )
else:
    print("\nA variável 'finetuned_model' não foi carregada. A avaliação foi abortada.")